In [1]:
from titrato import MicropKaTitrationCurve, plot_free_energy_vs_ph, plot_population_vs_ph, plot_micropka_network, tabulate_cycles, tabulate_free_energy
from IPython.display import Markdown
import matplotlib.pyplot as plt
import numpy as np

# Free energy analysis demonstration

This notebook demonstrates how to use the `titrato` library for analyzing pKa values and microscopic free energies.

## Loading data
Load a csv file containing the data using the `MicropKaTitrationCurve` class, which is intended for analyzing microscopic pKa values, one molecule at a time. It loads pKa values frome one file, and microstate charge information from another file.


### pKa file

The pKa file provides microscopic pKa values in CSV format (headers included):
Microstates are named with an identifier for the molecule, followed by an underscore, followed by a unique name for to identify the microscopic state.
Multiple molecules and their microstates can be included in a single file.

Example format:
```
Protonated,Deprotonated,pKa,SEM
SM01_micro001,SM01_micro005, -2.64,1.77
SM01_micro001,SM01_micro006,  7.67,1.77
SM01_micro004,SM01_micro002, 15.70,1.77
SM01_micro010,SM01_micro002, 10.54,1.77
SM01_micro005,SM01_micro004,  8.70,1.77
SM01_micro006,SM01_micro004, -1.61,1.77
SM01_micro008,SM01_micro004, -5.20,1.77
SM01_micro009,SM01_micro005, -6.55,1.77
SM01_micro005,SM01_micro010, 13.85,1.77
SM01_micro009,SM01_micro008,  7.35,1.77
```

### Charge file:
The charge file has listings of the molecular charge of each microstate. It has a CSV format (headers included) as follows
```csv
Molecule,Microstate ID,Charge
SM01,SM01_micro001,1
SM01,SM01_micro002,-2
SM01,SM01_micro004,-1
SM01,SM01_micro005,0
SM01,SM01_micro006,0
SM01,SM01_micro008,0
SM01,SM01_micro009,1
SM01,SM01_micro010,-1
```


In [2]:
# Run this cell for documentation on the titration curve representation
MicropKaTitrationCurve.from_id?

In [3]:
# Specify the molecule name, the data file location, the charge file location, and optional ph space (default is from 0-14)
curve = MicropKaTitrationCurve.from_id(
    "SM14",
    "./demo-pka-wuuvc-976-typeI-ECRISM-4.csv",
    "./demo-SAMPL6-microstate-charges.csv",
    ph_values=np.linspace(0, 14, 141),
)

## Cycle analysis

Obtain all cycles of specified length, and display as a markdown table.
Warning, the algorithm used is not well optimized for the molecules with larger networks.


In [4]:
# Obtain all cycles of lenght 4 
Markdown(tabulate_cycles(curve, length=4))

cycle | sum pKa 
 -----|----- 
  ['SM14_micro003', 'SM14_micro004', 'SM14_micro001', 'SM14_micro002'] | 0.000 
 ['SM14_micro003', 'SM14_micro002', 'SM14_micro001', 'SM14_micro004'] | 0.000 
 ['SM14_micro002', 'SM14_micro005', 'SM14_micro006', 'SM14_micro001'] | 0.000 
 ['SM14_micro002', 'SM14_micro001', 'SM14_micro006', 'SM14_micro005'] | -0.000 


Plotting the network is also possible.

In [5]:
# Plot each state, with pKa values on the arrows.
# Color indicates the charge of each state, see colorbar.
fig1, ax1 = plot_micropka_network(curve)


ModuleNotFoundError: No module named 'pydot'

<Figure size 525x825 with 0 Axes>

## Analyzing the free energy of each state.


The free energy of each state can be derived using the pKa values provided.
This script uses the most deprotonated state as the pKa reference.
However, by convention the free energy value for the lowest energy neutral state is 0 at every pH. This produces titration curves with intuitive slopes that match the charge of each species.

In [ ]:
Markdown(tabulate_free_energy(curve, base_e=False,ph=2.0))

In [ ]:
tabulate_free_energy(curve, base_e=False,ph=0.0)

The free energy as a function of pH can be plotted. 

In [ ]:
fig2, ax2 = plot_free_energy_vs_ph(curve,base_e=False, color_by_charge=False, mono_color=False)
leg2 = plt.legend(fontsize=12)

Populations can be derived from the free energy.

In [ ]:
fig3, ax3 = plot_population_vs_ph(curve,mono_color=False)
leg3 = plt.legend(fontsize=12)

Some prefer these plotted in log units.

In [ ]:
fig4, ax4 = plot_population_vs_ph(curve,mono_color=False)
plt.yscale("log")
leg4 = plt.legend(fontsize=12)